## Chapter 10: Web APIs with ASP.NET Core

So far, we’ve built a traditional web application where the server renders HTML views. But modern development often involves building **APIs** that serve data (usually JSON) to frontend clients like single‑page applications (React, Angular, Vue), mobile apps, or third‑party services. ASP.NET Core excels at building fast, scalable, and standards‑compliant HTTP APIs. In this chapter, you’ll learn how to design RESTful APIs, create API controllers, return appropriate HTTP responses, handle content negotiation, and document your API with Swagger. You’ll also see how to consume an API using `HttpClient`. By the end, you’ll be able to build a robust Web API that can power any client.

### 10.1 RESTful API Design Principles

**REST (Representational State Transfer)** is an architectural style for designing networked applications. It relies on a stateless, client‑server communication protocol—almost always HTTP. A RESTful API adheres to a set of principles:

- **Resources**: The key abstraction is a *resource*, which is any information that can be named (e.g., a product, a list of products). Resources are identified by URIs (e.g., `/api/products/123`).
- **HTTP verbs**: Operations on resources are performed using standard HTTP methods:
  - `GET` – retrieve a resource or collection.
  - `POST` – create a new resource.
  - `PUT` – replace an entire resource.
  - `PATCH` – partially update a resource.
  - `DELETE` – remove a resource.
- **Statelessness**: Each request from a client contains all the information the server needs to fulfill it. The server does not store any client context between requests. Authentication tokens are passed with each request.
- **Representations**: Clients interact with representations of resources (e.g., JSON, XML). The server can provide multiple representations of the same resource (content negotiation).
- **Hypermedia (HATEOAS)**: In a truly RESTful API, responses include links to navigate related resources. While this is the ideal, many practical APIs are “RESTful” without full HATEOAS.

**Example RESTful endpoints for a product resource:**

| Action               | HTTP Verb | URI                    | Status Code         |
|----------------------|-----------|------------------------|---------------------|
| Get all products     | GET       | /api/products          | 200 OK              |
| Get product by ID    | GET       | /api/products/5        | 200 OK or 404       |
| Create a new product | POST      | /api/products          | 201 Created         |
| Update entire product| PUT       | /api/products/5        | 204 NoContent or 200|
| Delete a product     | DELETE    | /api/products/5        | 204 NoContent       |

### 10.2 Creating an API Controller (`[ApiController]` attribute)

ASP.NET Core provides a dedicated set of features for building APIs, centered around the `[ApiController]` attribute.

#### Setting Up a Web API Project

You can create a new Web API project using the template:

```bash
dotnet new webapi -n MyApi
```

Or, in an existing MVC project, you can add API controllers alongside MVC controllers. The project template includes a sample `WeatherForecastController`.

#### Anatomy of an API Controller

```csharp
[ApiController]
[Route("api/[controller]")]
public class ProductsController : ControllerBase
{
    private readonly IProductService _productService;

    public ProductsController(IProductService productService)
    {
        _productService = productService;
    }

    [HttpGet]
    public async Task<ActionResult<IEnumerable<ProductDto>>> GetProducts()
    {
        var products = await _productService.GetAllProductsAsync();
        var productDtos = products.Select(p => new ProductDto { ... });
        return Ok(productDtos);
    }

    [HttpGet("{id}")]
    public async Task<ActionResult<ProductDto>> GetProduct(int id)
    {
        var product = await _productService.GetProductByIdAsync(id);
        if (product == null)
            return NotFound();

        return Ok(product);
    }

    [HttpPost]
    public async Task<ActionResult<ProductDto>> CreateProduct(CreateProductDto createDto)
    {
        var product = await _productService.CreateProductAsync(createDto);
        return CreatedAtAction(nameof(GetProduct), new { id = product.Id }, product);
    }

    [HttpPut("{id}")]
    public async Task<IActionResult> UpdateProduct(int id, UpdateProductDto updateDto)
    {
        if (id != updateDto.Id)
            return BadRequest();

        await _productService.UpdateProductAsync(updateDto);
        return NoContent();
    }

    [HttpDelete("{id}")]
    public async Task<IActionResult> DeleteProduct(int id)
    {
        await _productService.DeleteProductAsync(id);
        return NoContent();
    }
}
```

#### Key Features of `[ApiController]`

- **Automatic HTTP 400 responses**: If model validation fails, the framework automatically returns a `400 Bad Request` with details. You don’t need to check `ModelState.IsValid` manually (though you can still add custom errors).
- **Attribute routing requirement**: `[ApiController]` forces the use of attribute routing; conventional routes are not allowed.
- **Binding source inference**: The attribute infers complex parameter sources. For example, `[FromBody]` is assumed for complex types, `[FromForm]` for `IFormFile`, and `[FromRoute]`/`[FromQuery]` for simple types. You can override with attributes.
- **Problem details responses**: When an error occurs (e.g., 400), the response body is a `ProblemDetails` object, a standardized error format.

#### `ControllerBase` vs. `Controller`

For API controllers, derive from `ControllerBase`. It provides only the core methods for APIs (without view support), keeping the surface area clean.

### 10.3 Returning `IActionResult` and `ActionResult<T>`

API actions can return various types:

- **Specific type** (e.g., `ProductDto`): The framework automatically returns a `200 OK` with the serialized object. If the value is `null`, it returns `204 No Content`.
- **`IActionResult`**: Gives you full control (e.g., `return Ok(product)`, `return NotFound()`).
- **`ActionResult<T>`**: Combines the benefits of both. It allows you to return either a specific type or an action result, and provides better OpenAPI documentation.

```csharp
[HttpGet("{id}")]
public async Task<ActionResult<ProductDto>> GetProduct(int id)
{
    var product = await _productService.GetProductByIdAsync(id);
    if (product == null)
        return NotFound();   // returns ActionResult

    return product;           // implicitly converted to ActionResult<ProductDto>
}
```

#### Common Return Helpers

- `Ok(object?)` – 200 OK with optional body.
- `CreatedAtAction`, `CreatedAtRoute` – 201 Created with `Location` header.
- `NoContent()` – 204 No Content.
- `BadRequest()` – 400 Bad Request.
- `NotFound()` – 404 Not Found.
- `Unauthorized()` – 401 Unauthorized.
- `Forbid()` – 403 Forbidden.
- `Conflict()` – 409 Conflict (e.g., duplicate key).
- `Problem()` – Returns a `ProblemDetails` response (useful for custom errors).

#### Using DTOs (Data Transfer Objects)

Exposing your domain entities directly can lead to over‑posting and circular references. Instead, use **DTOs** to shape the data sent to clients.

```csharp
public class ProductDto
{
    public int Id { get; set; }
    public string Name { get; set; }
    public decimal Price { get; set; }
    public string CategoryName { get; set; }
}

public class CreateProductDto
{
    [Required]
    public string Name { get; set; }
    [Range(0.01, double.MaxValue)]
    public decimal Price { get; set; }
    public int CategoryId { get; set; }
}
```

Use AutoMapper to simplify mapping between entities and DTOs (optional but common).

### 10.4 Content Negotiation (JSON vs. XML)

**Content negotiation** is the process of selecting the best representation for a response based on the client's `Accept` header. By default, ASP.NET Core supports JSON (using `System.Text.Json`) and can be configured to support XML.

#### How It Works

The client sends an `Accept` header, e.g.:
```
Accept: application/json
```
or
```
Accept: application/xml
```

The framework looks at the registered formatters and picks the first that can produce the requested media type. If none match, it uses the default (JSON).

#### Adding XML Support

To enable XML, add the XML formatter in `Program.cs`:

```csharp
builder.Services.AddControllers()
    .AddXmlSerializerFormatters(); // or AddXmlDataContractSerializerFormatters
```

Now clients requesting XML will receive XML responses.

#### Forcing a Specific Format

You can override content negotiation by returning a `ContentResult` or using `[Produces]` attribute.

```csharp
[HttpGet("json-only")]
[Produces("application/json")]
public IActionResult GetJsonOnly() { ... }
```

### 10.5 Consuming an API with `HttpClient`

Building an API is only half the story; you often need to consume APIs from other services or client applications. `HttpClient` is the primary class for making HTTP requests in .NET.

#### Basic Usage

```csharp
using var httpClient = new HttpClient();
var response = await httpClient.GetAsync("https://api.example.com/products");
if (response.IsSuccessStatusCode)
{
    var json = await response.Content.ReadAsStringAsync();
    var products = JsonSerializer.Deserialize<List<ProductDto>>(json);
}
```

#### Best Practices with `HttpClient`

- **Do not** create a new `HttpClient` per request; it can exhaust socket resources. Instead, use `IHttpClientFactory` to manage `HttpClient` lifetimes.
- Register `HttpClient` with the factory in DI:

```csharp
builder.Services.AddHttpClient();
```

Then inject `IHttpClientFactory` into your service:

```csharp
public class ProductApiClient
{
    private readonly HttpClient _httpClient;

    public ProductApiClient(HttpClient httpClient)
    {
        _httpClient = httpClient;
    }

    public async Task<List<ProductDto>> GetProductsAsync()
    {
        return await _httpClient.GetFromJsonAsync<List<ProductDto>>("api/products");
    }
}
```

Configure a named or typed client for specific endpoints:

```csharp
builder.Services.AddHttpClient("ProductsApi", client =>
{
    client.BaseAddress = new Uri("https://api.example.com/");
    client.DefaultRequestHeaders.Add("User-Agent", "MyApp");
});
```

Then use `IHttpClientFactory.CreateClient("ProductsApi")` or inject a typed client.

### 10.6 Documenting APIs with Swagger/OpenAPI

**Swagger** (now the OpenAPI Specification) is the industry standard for documenting APIs. ASP.NET Core integrates with Swagger via the **Swashbuckle** package.

#### Installing Swashbuckle

Add the package:

```bash
dotnet add package Swashbuckle.AspNetCore
```

#### Configuring Swagger

In `Program.cs`:

```csharp
builder.Services.AddEndpointsApiExplorer();
builder.Services.AddSwaggerGen();

// ... after app = builder.Build()

if (app.Environment.IsDevelopment())
{
    app.UseSwagger();
    app.UseSwaggerUI();
}
```

Now navigate to `/swagger` to see the interactive documentation. Swagger analyzes your controllers, actions, and models to generate the OpenAPI document.

#### Enhancing Swagger Documentation

Use attributes to provide additional metadata:

- `[ProducesResponseType]` – describes possible response codes.
- `[Produces]` – specifies the response content type.
- `[Consumes]` – specifies the request content type.
- `[ApiConventionMethod]` – applies common conventions.

```csharp
[HttpGet("{id}")]
[ProducesResponseType(typeof(ProductDto), StatusCodes.Status200OK)]
[ProducesResponseType(StatusCodes.Status404NotFound)]
public async Task<ActionResult<ProductDto>> GetProduct(int id)
{
    // ...
}
```

You can also add XML comments to your code and configure Swagger to include them. First, enable XML documentation in the project file:

```xml
<PropertyGroup>
  <GenerateDocumentationFile>true</GenerateDocumentationFile>
  <NoWarn>$(NoWarn);1591</NoWarn>
</PropertyGroup>
```

Then in Swagger config:

```csharp
builder.Services.AddSwaggerGen(c =>
{
    var xmlFile = $"{Assembly.GetExecutingAssembly().GetName().Name}.xml";
    var xmlPath = Path.Combine(AppContext.BaseDirectory, xmlFile);
    c.IncludeXmlComments(xmlPath);
});
```

### 10.7 API Versioning (Brief)

As your API evolves, you may need to support multiple versions. ASP.NET Core provides the `Microsoft.AspNetCore.Mvc.Versioning` package.

#### Basic Setup

```bash
dotnet add package Microsoft.AspNetCore.Mvc.Versioning
```

```csharp
builder.Services.AddApiVersioning(options =>
{
    options.DefaultApiVersion = new ApiVersion(1, 0);
    options.AssumeDefaultVersionWhenUnspecified = true;
    options.ReportApiVersions = true;
});
```

#### Versioning Strategies

- **URL path versioning**: `/api/v1/products`, `/api/v2/products`
- **Query string versioning**: `/api/products?api-version=2.0`
- **Header versioning**: Custom header like `X-API-Version: 2.0`

You can specify the version on controllers or actions:

```csharp
[ApiVersion("1.0")]
[Route("api/v{version:apiVersion}/[controller]")]
public class ProductsController : ControllerBase
```

Or use the `[MapToApiVersion]` attribute to version specific actions.

### Summary

In this chapter, you’ve learned to build Web APIs with ASP.NET Core:

- **RESTful design** principles guide your endpoint structure.
- **`[ApiController]`** simplifies API development with automatic validation and binding.
- **Return types** like `ActionResult<T>` give flexibility and clear documentation.
- **Content negotiation** allows serving JSON, XML, or other formats.
- **`HttpClient`** and `IHttpClientFactory` enable consuming APIs efficiently.
- **Swagger/OpenAPI** provides interactive documentation.
- **Versioning** helps manage API evolution.

Web APIs are the backbone of modern distributed applications. In the next chapter, **"Authentication and Authorization,"** you’ll learn how to secure your APIs (and MVC apps) by implementing user identity, JWT tokens, and policy-based authorization—essential skills for protecting data and controlling access.

**Exercise:**
1. Create a new Web API project (or add an API controller to your existing project) for managing categories (`Category` entity with Id, Name, Description).
2. Implement CRUD endpoints for categories, returning appropriate status codes and using DTOs.
3. Add Swagger documentation and test your API using the Swagger UI.
4. In a separate console application, use `HttpClient` (with `IHttpClientFactory`) to consume your category API and list all categories.
5. (Optional) Implement API versioning (e.g., v1 returns basic info, v2 adds extra fields).